In [8]:
import json
import datetime
from web3 import Web3
import mysql.connector as mysql

#######
## INIT
#######

# load api key
secret = {}
with open('secret.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")

# init web3
infura_url = secret['INFURAURL1']
web3 = Web3(Web3.HTTPProvider(infura_url))
print(f"Connected to infura: {infura_url}")

Connected to infura: https://kovan.infura.io/v3/1070d8486bc64afdb9c730d6f2850d6c


In [9]:
# load abi
with open('abi_erc20.json') as f:
    abi = json.load(f)

In [10]:
wallet = '0xe9540071278Dba4374A42a84B48c5e230328AeF7'
token = '0xd6affe77bb9deaba1954ef90957100950c5e7f98'

In [11]:
contract = web3.eth.contract(abi=abi, address=Web3.toChecksumAddress(str(token)))

In [12]:
# token_symbol = str(contract.functions.symbol().call())
# token_name = str(contract.functions.name().call())
# print(token_symbol, token_name)

In [13]:
raw_balance = contract.functions.balanceOf(wallet).call()
balance = raw_balance/10**(contract.functions.decimals().call())
print(f"wallet {wallet} has {balance} tokens")

wallet 0xe9540071278Dba4374A42a84B48c5e230328AeF7 has 9998156.0 tokens


## define function to get balance

In [14]:
def get_wallet_erc20_balance(wallet, token):
    contract = web3.eth.contract(abi=abi, address=Web3.toChecksumAddress(str(token)))
    raw_balance = contract.functions.balanceOf(wallet).call()
    balance = raw_balance/10**(contract.functions.decimals().call())
    print(f"wallet {wallet} has {balance} tokens")
    return balance

# build rules logic

In [9]:
def select_users():
    db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
    cursor = db.cursor()
    query = f'SELECT account_id, user_ethereum_address FROM social_accounts;'
    cursor.execute(query)
    records = cursor.fetchall()
    cursor.close()
    db.close()
    #
    users = []
    for record in records:
        users.append(dict(zip(['discord_user_id', 'ethereum_address'], record)))
    return users

In [12]:
def update_user(account_id):
    # update blockchain_write_time
    db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
    cursor = db.cursor()
    query = "UPDATE social_accounts SET blockchain_check_time=%s WHERE account_id=%s;"
    values = (str(datetime.datetime.now()).split('.')[0], account_id)
    cursor.execute(query, values)
    db.commit()
    print(f"Updated {values[1]} to {values[0]}")
    cursor.close()
    db.close()  

In [10]:
# select rules
rules = select_rules(795011784913715240)

# select users
users = select_users()

# create list and dict for lookups
user_ids = [int(u['discord_user_id']) for u in users]
user_wallets = {int(u['discord_user_id']): u['ethereum_address'] for u in users}

In [26]:
member_ids = []
member_ids.append(789456739023192075)
member_ids.append(704521096837464076)
member_ids.append(519075961891979265)

In [28]:
for member_id in member_ids:
    if member_id in user_ids:
        print(f"User match {member_id} - begin roles process")
        
        # get wallet for this user
        wallet = user_wallets[member_id]
        print(f"Get balance for user")
        
        for rule in rules:
            # get user balance for this token
            balance = get_wallet_erc20_balance(wallet, rule['token_address'])
            
            # get rule ranges
            token_min = rule['token_min']
            token_max = rule['token_max']
            if token_max == None:
                token_max = 999999999999999999

            if balance >= token_min and balance <= token_max:
                print("assigning role")
                # assign role
                role = get(message.guild.roles, id=int(rule['role_id']))
                member.add_roles(role)
                print(f"assigned {role} to {member}")

User match 519075961891979265 - begin roles process


# bot sql queries

## select guilds

In [45]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT guild_id, guild_name, unite_setup_channel_id, created_at FROM discord_guilds;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
#
guilds = []
for record in records:
    guilds.append(dict(zip(['guild_id', 'guild_name', 'unite_setup_channel_id', 'created_at'], record)))

## select list of unite setup channels

In [47]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT distinct(unite_setup_channel_id) FROM discord_guilds;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
#
unite_setup_channels = []
for record in records:
    unite_setup_channels.append(record[0])

## insert guild

In [44]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = "INSERT INTO discord_guilds (guild_id, guild_name, unite_setup_channel_id, created_at) VALUES (%s, %s, %s, %s);"
values = (1337, "test_guild", 1234, str(datetime.datetime.now()).split('.')[0])
cursor.execute(query, values)
db.commit()
print(cursor.rowcount, "record inserted")
cursor.close()
db.close()

1 record inserted


## insert rule

In [79]:
guild_id = 1234
token_address = "0x1234"
token_min = 5
token_max = None
role_id = '795012514324807682'
role_name = "level 1"

In [80]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = "INSERT INTO discord_rules (guild_id, token_address, token_min, token_max, role_id, role_name, created_at) VALUES (%s, %s, %s, %s, %s, %s, %s);"
values = (guild_id, token_address, token_min, token_max, role_id, role_name, str(datetime.datetime.now()).split('.')[0])
cursor.execute(query, values)
db.commit()
print(cursor.rowcount, "record inserted")
cursor.close()
db.close()

1 record inserted


## select rules

In [16]:
guild_id = 795011784913715240

In [17]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT token_address, token_min, token_max, role_id, role_name FROM discord_rules where guild_id = {guild_id};'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
#
rules = []
for record in records:
    rule = dict(zip(['token_address', 'token_min', 'token_max', 'role_id', 'role_name'], record))
    rules.append(rule)

In [18]:
rules

[{'token_address': '0x2d7d8ba156897838eb738c50c3d56e2e5a5f7363',
  'token_min': 1,
  'token_max': 100,
  'role_id': '795012514324807682',
  'role_name': 'members'},
 {'token_address': '0x2d7d8ba156897838eb738c50c3d56e2e5a5f7363',
  'token_min': 100,
  'token_max': 150,
  'role_id': '1337',
  'role_name': 'top'}]

In [19]:
# get unique list of tokens
guild_token_addresses = [r['token_address'] for r in rules]
guild_token_addresses = list(set(token_addresses))

In [22]:
# get unique list of roles across rules
roles_for_rules = [r['role_id'] for r in rules]
roles_for_rules = list(set(roles_for_rules))
roles_for_rules

['1337', '795012514324807682']

## reset rules

In [81]:
guild_id = 1234

1234

In [83]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = "DELETE from discord_rules where guild_id = %s;"
values = (str(guild_id),)
cursor.execute(query, values)
db.commit()
print(cursor.rowcount, "record deleted")
cursor.close()
db.close()

1 record deleted


## select users

In [6]:
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT account_id, user_ethereum_address FROM social_accounts;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()
#
users = []
for record in records:
    users.append(dict(zip(['account_id', 'ethereum_address'], record)))

In [7]:
user_ids = [u['account_id'] for u in users]

In [8]:
user_ids

['396286449705680906', '519075961891979265']

In [13]:
users

[{'discord_user_id': '519075961891979265',
  'discord_user_name': 'Alex Conway#3235',
  'ethereum_address': '0x3708045C19c4D0eaB69289F8BC10D4a1fF4EDc6a'}]

## update user on discord_user_wallets table

In [12]:
discord_user_id = 519075961891979265

In [16]:
# update blockchain_write_time
db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = "UPDATE discord_user_wallets SET updated_at=%s WHERE discord_user_id=%s;"
values = (str(datetime.datetime.now()).split('.')[0], discord_user_id)
cursor.execute(query, values)
db.commit()
print(cursor.rowcount, "record updated")
cursor.close()
db.close()  

1 record updated


## select rules for guild

## get users to be updated

> get users with null check time or check time older than x hours

In [ ]:
# SELECT * FROM unite_staging.social_accounts WHERE (blockchain_check_time <= "2021-01-20 10:18:16" OR blockchain_check_time IS NULL);

In [17]:
from datetime import timedelta

In [20]:
datetime_check = datetime.datetime.now() - timedelta(hours=4)

db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
cursor = db.cursor()
query = f'SELECT account_id, user_ethereum_address FROM unite_staging.social_accounts WHERE (blockchain_check_time <= "{str(datetime_check)}" OR blockchain_check_time IS NULL);;'
cursor.execute(query)
records = cursor.fetchall()
cursor.close()
db.close()

users = []
for record in records:
    users.append(dict(zip(['account_id', 'ethereum_address'], record)))

In [30]:
def select_users_to_check(hours_ago=4):
    datetime_check = datetime.datetime.now() - timedelta(hours=hours_ago)

    db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
    cursor = db.cursor()
    query = f'SELECT account_id, user_ethereum_address FROM unite_staging.social_accounts WHERE (blockchain_check_time <= "{str(datetime_check)}" OR blockchain_check_time IS NULL);;'
    cursor.execute(query)
    records = cursor.fetchall()
    cursor.close()
    db.close()

    users = []
    for record in records:
        users.append(dict(zip(['account_id', 'ethereum_address'], record)))

    return users

# Build update process

In [34]:
# select users
users = select_users_to_check()

# create list and dict for lookups
user_ids = [int(u['account_id']) for u in users]
user_wallets = {int(u['account_id']): u['ethereum_address'] for u in users}

In [ ]:
# for member_id in member_ids:
#     if member_id in user_ids:
#         print(f"User match {member_id} - begin roles process")
        
#         # get wallet for this user
#         wallet = user_wallets[member_id]
#         print(f"Get balance for user")
        
#         for rule in rules:
#             # get user balance for this token
#             balance = get_wallet_erc20_balance(wallet, rule['token_address'])
            
#             # get rule ranges
#             token_min = rule['token_min']
#             token_max = rule['token_max']
#             if token_max == None:
#                 token_max = 999999999999999999

#             if balance >= token_min and balance <= token_max:
#                 print("assigning role")
#                 # assign role
#                 role = get(message.guild.roles, id=int(rule['role_id']))
#                 member.add_roles(role)
#                 print(f"assigned {role} to {member}")